In [ ]:
############## URUCHOM TO NAJPIERW! ###################

# zaimportuj wszystko i zdefiniuj funkcję wykonującą testy
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Ćwiczenie 1

Napisz metodę `parse` dla klasy `NetworkEnvelope`.

#### Wykonaj [ten test](/edit/r10/network.py): `network.py:NetworkEnvelopeTest:test_parse`

In [ ]:
# Ćwiczenie 1

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

### Ćwiczenie 2

Określ, co zawiera ten komunikat sieciowy:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [ ]:
# Ćwiczenie 2

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'

# skonwertuj na postać binarną
# Sprawdź bajty od 4 do 16

### Ćwiczenie 3

Napisz metodę `serialize` dla klasy `NetworkEnvelope`.

#### Wykonaj [ten test](/edit/r10/network.py): `network.py:NetworkEnvelopeTest:test_serialize`

In [ ]:
# Ćwiczenie 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

### Ćwiczenie 4

Napisz metodę `serialize` dla klasy `VersionMessage`.

#### Wykonaj [ten test](/edit/r10/network.py): `network.py:VersionMessageTest:test_serialize`

In [ ]:
# Ćwiczenie 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

### Ćwiczenie 5

Napisz metodę `handshake` dla klasy `SimpleNode`.

#### Wykonaj [ten test](/edit/r10/network.py): `network.py:SimpleNodeTest:test_handshake`

In [ ]:
# Ćwiczenie 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

### Ćwiczenie 6

Napisz metodę `serialize` dla klasy `GetHeadersMessage`.

#### Wykonaj [ten test](/edit/r10/network.py): `network.py:GetHeadersMessageTest:test_serialize`

In [ ]:
# Ćwiczenie 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

In [ ]:
from io import BytesIO
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import Block, GENESIS_BLOCK, LOWEST_BITS
from helper import calculate_new_bits
previous = Block.parse(BytesIO(GENESIS_BLOCK))
first_epoch_timestamp = previous.timestamp
expected_bits = LOWEST_BITS
count = 1
node = SimpleNode('mainnet.programmingbitcoin.com', testnet=False)
node.handshake()
for _ in range(19):
    getheaders = GetHeadersMessage(start_block=previous.hash())
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for header in headers.blocks:
        if not header.check_pow():
            raise RuntimeError('zły dowód pracy w bloku {}'.format(count))
        if header.prev_block != previous.hash():
            raise RuntimeError('nieciągły blok w {}'.format(count))
        if count % 2016 == 0:
            time_diff = previous.timestamp - first_epoch_timestamp
            expected_bits = calculate_new_bits(previous.bits, time_diff)
            print(expected_bits.hex())
            first_epoch_timestamp = header.timestamp
        if header.bits != expected_bits:
            raise RuntimeError('złe bity w bloku {}'.format(count))
        previous = header
        count += 1